# Solución: Métodos de Selección de Características

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import (
    VarianceThreshold, SelectKBest, f_regression, mutual_info_regression,
    SelectPercentile, RFE, RFECV, SelectFromModel
)

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error, r2_score

# Configuración para visualizaciones
plt.style.use('ggplot')
sns.set_theme(style="whitegrid")

# Ignorar advertencias
import warnings
warnings.filterwarnings('ignore')

## 1. Cargar y Explorar los Datos

In [ ]:
autos_df = pd.read_csv("../data/fe/autos.csv")

autos_df.head()

In [ ]:
# Información del dataset
autos_df.info()

In [ ]:
# Estadísticas descriptivas
autos_df.describe()

In [ ]:
# Verificar valores nulos
autos_df.isnull().sum()

## 2. Preparación de Datos

In [ ]:
# Identificar variables numéricas y categóricas
num_cols = autos_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = autos_df.select_dtypes(include=['object']).columns.tolist()

print(f"Variables numéricas: {len(num_cols)}")
print(f"Variables categóricas: {len(cat_cols)}")

In [ ]:
# Convertir variables categóricas a numéricas usando One-Hot Encoding
autos_encoded = pd.get_dummies(autos_df, columns=cat_cols, drop_first=True)
autos_encoded.shape

In [ ]:
# Definir las variables predictoras (X) y objetivo (y)
X = autos_encoded.drop('price', axis=1)
y = autos_encoded['price']

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Forma del conjunto de entrenamiento: {X_train.shape}")
print(f"Forma del conjunto de prueba: {X_test.shape}")

In [ ]:
# Escalar características numéricas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convertir de nuevo a DataFrame para mantener los nombres de columnas
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

## 3. Métodos de Filtrado

### 3.1 Eliminar Características Constantes y Cuasi-constantes

In [ ]:
# Eliminar características constantes
constant_filter = VarianceThreshold(threshold=0)
constant_filter.fit(X_train_scaled)

# Identificar características constantes
constant_features = [feature for feature, bool_val in zip(X_train.columns, constant_filter.get_support()) if not bool_val]
print(f"Características constantes: {constant_features}")
print(f"Número de características constantes: {len(constant_features)}")

In [ ]:
# Eliminar características cuasi-constantes (varianza < 0.01)
quasi_constant_filter = VarianceThreshold(threshold=0.01)
quasi_constant_filter.fit(X_train_scaled)

# Identificar características cuasi-constantes
quasi_constant_features = [feature for feature, bool_val in zip(X_train.columns, quasi_constant_filter.get_support()) if not bool_val]
print(f"Características cuasi-constantes: {quasi_constant_features}")
print(f"Número de características cuasi-constantes: {len(quasi_constant_features)}")

In [ ]:
# Eliminar características constantes y cuasi-constantes
X_train_no_const = X_train_scaled.drop(columns=quasi_constant_features)
X_test_no_const = X_test_scaled.drop(columns=quasi_constant_features)

print(f"Número original de características: {X_train_scaled.shape[1]}")
print(f"Número de características después de eliminar constantes y cuasi-constantes: {X_train_no_const.shape[1]}")

### 3.2 Selección Univariada con SelectKBest y SelectPercentile

In [ ]:
# SelectKBest con f_regression
k_best_f = SelectKBest(f_regression, k=10)
X_train_kbest_f = k_best_f.fit_transform(X_train_no_const, y_train)

# Obtener las características seleccionadas
selected_features_kbest_f = X_train_no_const.columns[k_best_f.get_support()]
print("Top 10 características seleccionadas con f_regression:")
print(selected_features_kbest_f.tolist())

In [ ]:
# SelectKBest con mutual_info_regression
k_best_mi = SelectKBest(mutual_info_regression, k=10)
X_train_kbest_mi = k_best_mi.fit_transform(X_train_no_const, y_train)

# Obtener las características seleccionadas
selected_features_kbest_mi = X_train_no_const.columns[k_best_mi.get_support()]
print("Top 10 características seleccionadas con mutual_info_regression:")
print(selected_features_kbest_mi.tolist())

In [ ]:
# SelectPercentile con f_regression (20% superior)
percentile_selector = SelectPercentile(f_regression, percentile=20)
X_train_percentile = percentile_selector.fit_transform(X_train_no_const, y_train)

# Obtener las características seleccionadas
selected_features_percentile = X_train_no_const.columns[percentile_selector.get_support()]
print(f"Características seleccionadas en el percentil 20 superior:")
print(selected_features_percentile.tolist())
print(f"Número de características seleccionadas: {len(selected_features_percentile)}")

### 3.3 Análisis de Correlación con Mapa de Calor

In [ ]:
# Calcular la matriz de correlación
corr_matrix = autos_df.corr(numeric_only=True)

# Visualizar el mapa de calor (solo para las top 15 características más correlacionadas con el precio)
plt.figure(figsize=(12, 10))
correlation_with_price = corr_matrix['price'].sort_values(ascending=False)[1:16]  # Excluir price mismo
top_corr_features = correlation_with_price.index.tolist()

# Crear un subset de la matriz de correlación con las características más correlacionadas
top_corr_matrix = corr_matrix.loc[top_corr_features, top_corr_features]

# Crear el mapa de calor
sns.heatmap(top_corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Mapa de Calor de Correlación para las 15 Características Más Correlacionadas')
plt.tight_layout()
plt.show()

# Mostrar las 15 características más correlacionadas con el precio
print("15 características más correlacionadas con el precio:")
for feature, corr in correlation_with_price.items():
    print(f"{feature}: {corr:.4f}")

In [ ]:
# Identificar pares de características altamente correlacionadas entre sí
high_corr_pairs = []

for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.9:
            high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))

# Mostrar pares de características altamente correlacionadas
print(f"Número de pares de características altamente correlacionadas: {len(high_corr_pairs)}")
for feature1, feature2, corr in high_corr_pairs:
    print(f"{feature1} - {feature2}: {corr:.4f}")

In [ ]:
# Eliminar características altamente correlacionadas (manteniendo la más correlacionada con el precio)
def remove_correlated_features(X, threshold=0.9):
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Identificar características para eliminar
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    
    # Para cada par de características correlacionadas, mantener la más correlacionada con el precio
    price_corr = X.corrwith(y).abs()
    features_to_keep = []
    
    for i in range(len(upper.columns)):
        for j in range(i+1, len(upper.columns)):
            col_i = upper.columns[i]
            col_j = upper.columns[j]
            if upper.iloc[i, j] > threshold:
                if price_corr[col_i] > price_corr[col_j]:
                    if col_j in to_drop and col_j not in features_to_keep:
                        features_to_keep.append(col_i)
                else:
                    if col_i in to_drop and col_i not in features_to_keep:
                        features_to_keep.append(col_j)
    
    # Eliminar características, pero mantener las que están en features_to_keep
    final_to_drop = [col for col in to_drop if col not in features_to_keep]
    
    return X.drop(columns=final_to_drop), final_to_drop

X_train_no_corr, dropped_features = remove_correlated_features(X_train_no_const)

print(f"Número de características eliminadas por alta correlación: {len(dropped_features)}")
print(f"Características eliminadas: {dropped_features}")
print(f"Número de características restantes: {X_train_no_corr.shape[1]}")

## 4. Métodos Wrapper

### 4.1 Eliminación Recursiva de Características (RFE)

In [ ]:
# Aplicar RFE con regresión lineal
lm = LinearRegression()
rfe = RFE(estimator=lm, n_features_to_select=10, step=1)
rfe.fit(X_train_no_corr, y_train)

# Obtener las características seleccionadas
selected_features_rfe = X_train_no_corr.columns[rfe.get_support()]

print("Características seleccionadas por RFE:")
print(selected_features_rfe.tolist())

In [ ]:
# Evaluar el modelo con las características seleccionadas por RFE
X_train_rfe = X_train_no_corr[selected_features_rfe]
X_test_rfe = X_test_scaled[selected_features_rfe]

lm.fit(X_train_rfe, y_train)
y_pred_rfe = lm.predict(X_test_rfe)

rmse_rfe = np.sqrt(mean_squared_error(y_test, y_pred_rfe))
r2_rfe = r2_score(y_test, y_pred_rfe)

print(f"RMSE con características RFE: {rmse_rfe:.2f}")
print(f"R² con características RFE: {r2_rfe:.4f}")

In [ ]:
# Aplicar RFECV para encontrar el número óptimo de características
rfecv = RFECV(estimator=lm, step=1, cv=5, scoring='neg_mean_squared_error')
rfecv.fit(X_train_no_corr, y_train)

# Obtener las características seleccionadas
selected_features_rfecv = X_train_no_corr.columns[rfecv.get_support()]

print(f"Número óptimo de características: {rfecv.n_features_}")
print("Características seleccionadas por RFECV:")
print(selected_features_rfecv.tolist())

In [ ]:
# Evaluar el modelo con las características seleccionadas por RFECV
X_train_rfecv = X_train_no_corr[selected_features_rfecv]
X_test_rfecv = X_test_scaled[selected_features_rfecv]

lm.fit(X_train_rfecv, y_train)
y_pred_rfecv = lm.predict(X_test_rfecv)

rmse_rfecv = np.sqrt(mean_squared_error(y_test, y_pred_rfecv))
r2_rfecv = r2_score(y_test, y_pred_rfecv)

print(f"RMSE con características RFECV: {rmse_rfecv:.2f}")
print(f"R² con características RFECV: {r2_rfecv:.4f}")

## 5. Métodos Integrados

### 5.1 Selección con LASSO

In [ ]:
# Aplicar Lasso para selección de características
lasso = Lasso(alpha=0.1)  # Ajustar alpha según sea necesario
lasso.fit(X_train_no_corr, y_train)

# Mostrar los coeficientes
coefs = pd.Series(lasso.coef_, index=X_train_no_corr.columns)
importance = pd.DataFrame({'Feature': X_train_no_corr.columns, 'Importance': np.abs(lasso.coef_)})
importance = importance.sort_values('Importance', ascending=False)

# Visualizar las 15 características más importantes según LASSO
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=importance.head(15))
plt.title('Top 15 Características Seleccionadas por LASSO')
plt.tight_layout()
plt.show()

In [ ]:
# Evaluar el modelo con las características seleccionadas por LASSO
lm.fit(X_train_lasso, y_train)
y_pred_lasso = lm.predict(X_test_lasso)

rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))
r2_lasso = r2_score(y_test, y_pred_lasso)

print(f"RMSE con características LASSO: {rmse_lasso:.2f}")
print(f"R² con características LASSO: {r2_lasso:.4f}")

In [ ]:
# Seleccionar características con coeficientes no nulos
lasso_selector = SelectFromModel(lasso, prefit=True)
X_train_lasso = lasso_selector.transform(X_train_no_corr)
X_test_lasso = lasso_selector.transform(X_test_scaled[X_train_no_corr.columns])

# Obtener nombres de características seleccionadas
selected_features_lasso = X_train_no_corr.columns[lasso_selector.get_support()]

print(f"Número de características seleccionadas por LASSO: {len(selected_features_lasso)}")
print("Características seleccionadas:")
print(selected_features_lasso.tolist())

In [ ]:
# Probar con diferentes valores de alpha para LASSO
alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
rmse_scores = []
r2_scores = []
num_features = []

for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train_no_corr, y_train)
    
    # Contar características no nulas
    n_features = sum(lasso.coef_ != 0)
    num_features.append(n_features)
    
    # Evaluar en el conjunto de prueba
    y_pred = lasso.predict(X_test_scaled[X_train_no_corr.columns])
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    rmse_scores.append(rmse)
    r2_scores.append(r2)
    
    print(f"Alpha: {alpha}, # Features: {n_features}, RMSE: {rmse:.2f}, R²: {r2:.4f}")

In [ ]:
# Visualizar el efecto de alpha en LASSO
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.semilogx(alphas, rmse_scores, marker='o')
plt.xlabel('Alpha')
plt.ylabel('RMSE')
plt.title('RMSE vs Alpha')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.semilogx(alphas, num_features, marker='o')
plt.xlabel('Alpha')
plt.ylabel('Número de Características')
plt.title('Número de Características vs Alpha')
plt.grid(True)

plt.tight_layout()
plt.show()

## 6. Comparación de Modelos y Conclusión

In [ ]:
# Comparar todos los métodos de selección de características
models = {
    'RFE (10 feat.)': {'features': selected_features_rfe, 'rmse': rmse_rfe, 'r2': r2_rfe},
    'RFECV': {'features': selected_features_rfecv, 'rmse': rmse_rfecv, 'r2': r2_rfecv},
    'LASSO': {'features': selected_features_lasso.tolist(), 'rmse': rmse_lasso, 'r2': r2_lasso}
}

# Entrenar modelo con todas las características disponibles (después de eliminar correlacionadas)
lm.fit(X_train_no_corr, y_train)
y_pred_all = lm.predict(X_test_scaled[X_train_no_corr.columns])
rmse_all = np.sqrt(mean_squared_error(y_test, y_pred_all))
r2_all = r2_score(y_test, y_pred_all)

models['Todas las características'] = {
    'features': X_train_no_corr.columns.tolist(), 
    'rmse': rmse_all, 
    'r2': r2_all
}

# Crear un DataFrame para comparar resultados
comparison = pd.DataFrame({
    'Método': list(models.keys()),
    'Número de Características': [len(model['features']) for model in models.values()],
    'RMSE': [model['rmse'] for model in models.values()],
    'R²': [model['r2'] for model in models.values()]
})

comparison.sort_values('RMSE')

In [ ]:
# Visualizar la comparación
plt.figure(figsize=(14, 10))

plt.subplot(2, 1, 1)
sns.barplot(x='Método', y='RMSE', data=comparison)
plt.title('RMSE por Método de Selección de Características')
plt.xticks(rotation=45)
plt.grid(True, axis='y')

plt.subplot(2, 1, 2)
sns.barplot(x='Método', y='Número de Características', data=comparison)
plt.title('Número de Características por Método de Selección')
plt.xticks(rotation=45)
plt.grid(True, axis='y')

plt.tight_layout()
plt.show()

## 7. Características Finales Seleccionadas

Basado en los resultados anteriores, seleccionamos el conjunto de características del método que dio el mejor balance entre rendimiento (RMSE/R²) y simplicidad (número de características).

In [ ]:
# Determinar el mejor método (en este caso asumimos que es RFECV)
best_method = 'LASSO'
best_features = models[best_method]['features']

print(f"Método seleccionado: {best_method}")
print(f"Número de características: {len(best_features)}")
print(f"RMSE: {models[best_method]['rmse']:.2f}")
print(f"R²: {models[best_method]['r2']:.4f}")
print("\nCaracterísticas seleccionadas:")
for feature in best_features:
    print(f"- {feature}")

In [ ]:
# Guardar la lista de características seleccionadas
selected_features_df = pd.DataFrame({'feature': best_features})

selected_features_df.to_csv('selected_features.csv', index=False)

# Guardar también el dataset con solo las características seleccionadas
final_df = autos_df.copy()
final_df = pd.get_dummies(final_df, columns=cat_cols, drop_first=True)
final_df = final_df[list(best_features) + ['price']]
final_df.to_csv('autos_selected_features.csv', index=False)